<a href="https://colab.research.google.com/github/Shubham-M-Rathod/CodeAssistant/blob/main/CodeAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Single LLM Based

In [ ]:
!pip install -q transformers sentencepiece accelerate bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, SummarizationPipeline

pipeline = SummarizationPipeline(
    model=AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-base-multi-sum"),
    tokenizer=AutoTokenizer.from_pretrained("Salesforce/codet5-base-multi-sum", skip_special_tokens=True, use_fast=False, padding='max_length')
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

In [ ]:
import ast

def find_and_summarize_python(code):
  ans=''
  root = ast.parse(code)
  for node in ast.walk(root):
      if isinstance(node, ast.FunctionDef):
          function_code = ast.get_source_segment(code, node)
          print(f"Found function: {node.name}")
          summary = summarize_function(function_code)
          print(f"Summary: {summary}\n")
          ans += f"{node.name}: {summary[0].get('summary_text')}\n"
  return ans

def summarize_function(function_code):
  summaries = pipeline(function_code, max_length=30)
  return summaries

In [ ]:
example=[
    """def get_vectorstore(text):
      embeddings = VertexAIEmbeddings()
      vectorstore = FAISS.from_texts(texts=text, embedding=embeddings)
      return vectorstore
      def get_chat(vectorstore):
        llm = VertexAI(temperature=0.5)
        memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
        conv = ConversationalRetrievalChain.from_llm(llm=llm, retriever=vectorstore.as_retriever(), memory=memory)
        return conv""",
    """def sequential_model():
        model = Sequential()
        model.add(Dense(384, activation="tanh"))
        model.add(Dense(128, activation="tanh"))
        model.add(Dense(32, activation="tanh"))
        model.add(Dense(8, activation="tanh"))
        model.add(Dense(1, activation="sigmoid"))
        return model
def predict_output(x):
        model = sequential_model()
        pred = model.predict(x)
        return pred"""]
print(find_and_summarize_python(example[0]))

Found function: get_vectorstore
Summary: [{'summary_text': 'Get the vectorstore from the given text .'}]

Found function: get_chat
Summary: [{'summary_text': 'Get a conversational retrieval chain for the given vectorstore .'}]

get_vectorstore: Get the vectorstore from the given text .
get_chat: Get a conversational retrieval chain for the given vectorstore .



In [ ]:
! pip install javalang -q

In [ ]:
import javalang as jl

def __get_start_end_for_node(node_to_find, tree):
    start = None
    end = None
    for path, node in tree:
        if start is not None and node_to_find not in path:
            end = node.position
            return start, end
        if start is None and node == node_to_find:
            start = node.position
    return start, end


def __get_string(start, end, code):
    if start is None:
        return ""

    end_pos = None

    if end is not None:
        end_pos = end.line - 1

    lines = code.splitlines(True)
    string = "".join(lines[start.line:end_pos])
    string = lines[start.line - 1] + string

    if end is None:
        left = string.count("{")
        right = string.count("}")
        if right - left == 1:
            p = string.rfind("}")
            string = string[:p]

    return string

def find_and_summarize_java(code):
  ans=''
  tree = jl.parse.parse(code)
  for _, node in tree.filter(jl.tree.MethodDeclaration):
      start, end = __get_start_end_for_node(node, tree)
      function_code = __get_string(start, end, code)
      summary = summarize_function(function_code)
      print(f"Summary: {summary}\n")
      ans += f"{node.name}: {summary[0].get('summary_text')}\n"
  return ans

In [ ]:
example=[
    """import java.util.Arrays;

class GFG {
    public static void main(String args[])
    {
        int[] arr = { 5, -2, 23, 7, 87, -42, 509 };
        Arrays.sort(arr);
        printArray(arr);
    }

    static void printArray(int arr[])
    {
      System.out.println("\nThe sorted array is: ");
        for (int num : arr) {
            System.out.print(num + " ");
        }
      }
}"""]

In [ ]:
print(find_and_summarize_java(example[0]))

Summary: [{'summary_text': 'This is a test program that prints an array of int with the names of the terms in the grammar .'}]

Summary: [{'summary_text': 'This method prints an array of integers in the order of the element in the order of the element in the order of the element in the order'}]

main: This is a test program that prints an array of int with the names of the terms in the grammar .
printArray: This method prints an array of integers in the order of the element in the order of the element in the order of the element in the order



In [ ]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


###### Streamlit

In [ ]:
#
%%writefile app.py
import streamlit as st
import ast
import javalang

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, SummarizationPipeline

pipeline = SummarizationPipeline(
    model=AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-base-multi-sum"),
    tokenizer=AutoTokenizer.from_pretrained("Salesforce/codet5-base-multi-sum", skip_special_tokens=True, use_fast=False, padding='max_length')
)

def find_and_summarize_python(code):
  ans=''
  root = ast.parse(code)
  for node in ast.walk(root):
      if isinstance(node, ast.FunctionDef):
          function_code = ast.get_source_segment(code, node)
          print(f"Found function: {node.name}")
          summary = summarize_function(function_code)
          print(f"Summary: {summary}\n")
          ans += f"{node.name}: {summary[0].get('summary_text')}\n"
  return ans

import javalang as jl

def __get_start_end_for_node(node_to_find, tree):
    start = None
    end = None
    for path, node in tree:
        if start is not None and node_to_find not in path:
            end = node.position
            return start, end
        if start is None and node == node_to_find:
            start = node.position
    return start, end


def __get_string(start, end, code):
    if start is None:
        return ""

    end_pos = None

    if end is not None:
        end_pos = end.line - 1

    lines = code.splitlines(True)
    string = "".join(lines[start.line:end_pos])
    string = lines[start.line - 1] + string

    if end is None:
        left = string.count("{")
        right = string.count("}")
        if right - left == 1:
            p = string.rfind("}")
            string = string[:p]

    return string

def find_and_summarize_java(code):
  ans=''
  tree = jl.parse.parse(code)
  for _, node in tree.filter(jl.tree.MethodDeclaration):
      start, end = __get_start_end_for_node(node, tree)
      function_code = __get_string(start, end, code)
      summary = summarize_function(function_code)
      print(f"Summary: {summary}\n")
      ans += f"{node.name}: {summary[0].get('summary_text')}\n"
  return ans

def summarize_function(function_code):
  summaries = pipeline(function_code, max_length=30)
  return summaries

st.title('Enter Code')
language = st.selectbox('Select Language', ('Python', 'Java'))

with st.form(key='my_form'):
    code_input = st.text_area('Enter Code', value='', height=200)
    submit_button = st.form_submit_button(label='Submit')

if submit_button:
    if language == 'Python':
      summary = find_and_summarize_python(code_input)
    elif language == 'Java':
      summary = find_and_summarize_java(code_input)
    st.write(summary)

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 --subdomain myapp

### Multi-Agent LLM Based

In [1]:
!pip install pyautogen 'pyautogen[retrievechat]' chromadb pypdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
import os
import autogen
import chromadb
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent

In [3]:
os.environ['OAI_CONFIG_LIST'] = """
[{"model": "google/gemma-7b-it",
"api_key": "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",
"base_url": "https://openrouter.ai/api/v1",
"max_tokens":1000}]
"""

llm_config = {
    "timeout": 600,
    "cache_seed": 28,
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["google/gemma-7b-it"]},
    ),
    "temperature": 0.1,
}


In [4]:
# assistant = autogen.AssistantAgent(
#     name='Coder',
#     llm_config=llm_config,
#     system_message='You are a python coding assistant and write scripts and scrape the web to execute daily tasks'
#   )

In [5]:
# user_assistant = autogen.UserProxyAgent(
#     name='User_Coder',
#     human_input_mode='NEVER',
#     max_consecutive_auto_reply=10,
#     is_termination_msg=lambda x:x.get('content','').rstrip().endswith('TERMINATE'),
#     code_execution_config={
#         'use_docker':False,
#     },
#   )

In [6]:
source = 'https://raw.githubusercontent.com/Shubham-M-Rathod/ecommerce_mern/main/my_api/controllers/product_controller.js'

In [7]:
manager_contexted = RetrieveUserProxyAgent(
    name='manager_contexted',
    is_termination_msg=lambda x:x.get('content','').rstrip().endswith('TERMINATE'),
    system_message='manager who has extra context retrieval power and asks questions. reply `TERMINATE` if task done',
    human_input_mode='NEVER',
    max_consecutive_auto_reply=5,
    retrieve_config={
        'docs_path':source,
        'chunk_token_size':1000,
        'model':'google/gemma-7b-it',
        'client':chromadb.PersistentClient(path='tmp/chromadb'),
        'collection_name':'groupchat',
        'get_or_create':True,
    },
    code_execution_config=False
  )

In [8]:
coder = AssistantAgent(
    name='documenter',
    is_termination_msg=lambda x:x.get('content','').rstrip().endswith('TERMINATE'),
    system_message='You are a experienced programmer. Write code summary',
    llm_config=llm_config
  )

In [9]:
reviewer = AssistantAgent(
    name='Reviewer',
    is_termination_msg=lambda x:x.get('content','').rstrip().endswith('TERMINATE'),
    system_message='You are a senior programmer. Write code review. Reply `TERMINATE` if task done',
    llm_config=llm_config
  )

In [66]:
groupchat = autogen.GroupChat(
    agents=[manager_contexted, coder, reviewer],
    messages=[],
    max_round=3,
    speaker_selection_method='auto',
    allow_repeat_speaker=False
  )

In [67]:
groupManager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [68]:
problem = 'Write code documentation for the piece of code'

In [69]:
# def get_file_from_url(url: str, save_path: str = None):
#     """Download a file from a URL."""
#     if save_path is None:
#         current_dir = os.getcwd()
#         temp_dir = os.path.join(current_dir, "chromadb")
#         save_path = os.path.join(temp_dir, os.path.basename(url))
#         os.makedirs(os.path.dirname(save_path), exist_ok=True)

#     with requests.get(url, stream=True) as r:
#         r.raise_for_status()
#         with open(save_path, "wb") as f:
#             for chunk in r.iter_content(chunk_size=8192):
#                 f.write(chunk)
#     return save_path

In [70]:
manager_contexted.reset()
manager_contexted.initiate_chat(groupManager, problem=problem)

doc_ids:  [['doc_0']]
Adding doc_id doc_0 to context.
manager_contexted (to chat_manager):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user. You should follow the following steps to answer a question:
Step 1, you estimate the user's intent based on the question and context. The intent can be a code generation task or
a question answering task.
Step 2, you reply based on the intent.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
If user's intent is code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

If user's intent is question answering, you must give as short an answer as possible.

User's question is: Write code documentation for the piece of code

Conte

**Role:** Documenter

**Summary:**

The text describes a code snippet and provides a list of functions and their descriptions. It also includes information about the context and the steps to answer a question. The code is written in JavaScript and is related to an e-commerce website.

**Key Points:**

* The code defines several functions for managing products, including `getProductById`, `getProducts`, `new_product`, and `deleteProduct`.
* The functions are exported as part of an object called `exports`.
* The code uses the `ProductSchema` model to interact with the database.
* The code relies on the `fs` module to delete files.
* The code includes documentation for each function, including its purpose, parameters, and return values.


documenter (to chat_manager):

## Code Summary

This code defines a Node.js module named `manager_contexted` that provides endpoints for managing products in an online store. It uses the Mongoose framework to interact with a MongoDB database.

**Key functionalities:**

* **Product retrieval:**
    * `getProductById`: Retrieves a product by its name from the database.
    * `getProducts`: Retrieves all products from the database.
* **Product creation:**
    * `new_product`: Creates a new product in the database.
* **Product deletion:**
    * `deleteProduct`: Deletes a product from the database.

**Additional notes:**

* The code assumes that the `ProductSchema` model is defined in the `../model/product_model` file.
* The code uses the `fs` module to delete the product image file from the server.
* The code exports several functions for each endpoint, allowing for modularity.

**Overall, this code provides a well-structured and functional solution for managing products in an online store

## Role: Reviewer

**Review:**

This code defines a Node.js module named `manager_contexted` that provides endpoints for managing products in an online store. The code is well-structured and easy to understand, although the use of the `fs` module to delete product images raises some concerns.

**Strengths:**

* **Modularity:** The code exports several functions for each endpoint, allowing for modularity and reusability.
* **Functionality:** The code covers a wide range of functionalities, including product retrieval, creation, and deletion.
* **Database interaction:** The code uses the Mongoose framework to interact with the MongoDB database efficiently.

**Areas for improvement:**

* **Security:** The code does not include any security measures, such as authentication or authorization.
* **Error handling:** The code does not handle errors gracefully. For example, the `getProductById` function does not handle the case where the product is not found.
* **Image deletion:** The use of the

Reviewer (to chat_manager):

## Code Review

**Review:**

This code defines a Node.js module named `manager_contexted` that provides endpoints for managing products in an online store. It uses the Mongoose framework to interact with a MongoDB database.

**Strengths:**

* **Well-structured:** The code is well-organized into functions for each endpoint, making it easy to understand and maintain.
* **Modular:** The code exports several functions for each endpoint, allowing for modularity and reusability.
* **Database interaction:** The code uses Mongoose to interact with the MongoDB database efficiently.

**Areas for improvement:**

* **Error handling:** The code does not handle errors gracefully. For example, the `getProductById` function does not handle the case where the product is not found.
* **Logging:** The code does not include any logging functionality. This could make it difficult to debug errors.
* **Security:** The code does not include any security measures. For example, the 

ChatResult(chat_id=None, chat_history=[{'content': 'You\'re a retrieve augmented chatbot. You answer user\'s questions based on your own knowledge and the\ncontext provided by the user. You should follow the following steps to answer a question:\nStep 1, you estimate the user\'s intent based on the question and context. The intent can be a code generation task or\na question answering task.\nStep 2, you reply based on the intent.\nIf you can\'t answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.\nIf user\'s intent is code generation, you must obey the following rules:\nRule 1. You MUST NOT install any packages because all the packages needed are already installed.\nRule 2. You must follow the formats below to write your code:\n```language\n# your code\n```\n\nIf user\'s intent is question answering, you must give as short an answer as possible.\n\nUser\'s question is: Write code documentation for the piece of code\n\nContext is: const Pro

In [71]:
for i in groupchat.messages:
   if i['name']=='documenter':
    print(i['content'])
    print('---'*30)

## Code Summary

This code defines a Node.js module named `manager_contexted` that provides endpoints for managing products in an online store. It uses the Mongoose framework to interact with a MongoDB database.

**Key functionalities:**

* **Product retrieval:**
    * `getProductById`: Retrieves a product by its name from the database.
    * `getProducts`: Retrieves all products from the database.
* **Product creation:**
    * `new_product`: Creates a new product in the database.
* **Product deletion:**
    * `deleteProduct`: Deletes a product from the database.

**Additional notes:**

* The code assumes that the `ProductSchema` model is defined in the `../model/product_model` file.
* The code uses the `fs` module to delete the product image file from the server.
* The code exports several functions for each endpoint, allowing for modularity.

**Overall, this code provides a well-structured and functional solution for managing products in an online store.**
---------------------------

###### Streamlit

In [58]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


In [75]:
%%writefile app.py
import streamlit as st
import ast
import os
import autogen
import chromadb
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
os.environ['OAI_CONFIG_LIST'] = """
[{"model": "google/gemma-7b-it",
"api_key": "sk-or-v1-",
"base_url": "https://openrouter.ai/api/v1",
"max_tokens":1000}]
"""

llm_config = {
    "timeout": 600,
    "cache_seed": 28,
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["google/gemma-7b-it"]},
    ),
    "temperature": 0.1,
}
coder = AssistantAgent(
    name='documenter',
    is_termination_msg=lambda x:x.get('content','').rstrip().endswith('TERMINATE'),
    system_message='You are a experienced programmer. Write code summary',
    llm_config=llm_config
  )
reviewer = AssistantAgent(
    name='Reviewer',
    is_termination_msg=lambda x:x.get('content','').rstrip().endswith('TERMINATE'),
    system_message='You are a senior programmer. Write code review. Reply `TERMINATE` if task done',
    llm_config=llm_config
  )

st.title('Enter Github Source Code Link (Raw)')

with st.form(key='my_form'):
    source = st.text_area('Enter Code', value='', height=50)
    submit_button = st.form_submit_button(label='Submit')

if submit_button:
  progress_bar = st.progress(0)
  manager_contexted = RetrieveUserProxyAgent(
      name='manager_contexted',
      is_termination_msg=lambda x:x.get('content','').rstrip().endswith('TERMINATE'),
      system_message='manager who has extra context retrieval power and asks questions. reply `TERMINATE` if task done',
      human_input_mode='NEVER',
      max_consecutive_auto_reply=5,
      retrieve_config={
          'docs_path':source,
          'chunk_token_size':1000,
          'model':'google/gemma-7b-it',
          'client':chromadb.PersistentClient(path='tmp/chromadb'),
          'collection_name':'groupchat',
          'get_or_create':True,
      },
      code_execution_config=False
    )
  groupchat = autogen.GroupChat(
    agents=[manager_contexted, coder, reviewer],
    messages=[],
    max_round=5,
    speaker_selection_method='auto',
    allow_repeat_speaker=False
  )
  groupManager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)
  problem = 'Write code documentation for the piece of code'
  manager_contexted.reset()
  progress_bar.progress(70)
  manager_contexted.initiate_chat(groupManager, problem=problem)
  progress_bar.progress(100)

  for i in groupchat.messages:
    if i['name']=='documenter':
      st.write(i['content'])

Overwriting app.py


In [76]:
!curl https://loca.lt/mytunnelpassword

34.27.36.224

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 --subdomain myapp




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.27.36.224:8501

npx: installed 22 in 2.013s
your url is: https://myapp.loca.lt
Trying to create collection.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Number of requested results 20 is greater than number of elements in index 1, updating n_results = 1
doc_ids:  [['doc_0']]
Model google/gemma-7b-it not found. Using cl100k_base encoding.
Adding doc_id doc_0 to context.
manager_contexted (to chat_manager):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided b